In [15]:
# パッケージをインストールしていない場合
# !pip install eth_account
# !pip install web3

In [6]:
from eth_account import Account # Ethereumのアカウントを管理するライブラリ
from web3 import Web3, HTTPProvider # Web3.pyライブラリ
import secrets
import json
web3 = Web3(HTTPProvider("http://153.122.10.129:8545")) # web3のオブジェクト
private_key = "0x" + secrets.token_hex(32) # 秘密鍵の生成
address = Account.privateKeyToAccount(private_key).address # アドレス
bytecode = json.load(open("bytecode.json", 'r'))['object'] # バイトコード
MyToken = web3.eth.contract(abi=[], bytecode=bytecode) 
# コントラクト実装用のトランザクションの作成
construct_tx = MyToken.constructor().buildTransaction({
        'from': address, # 送信元アドレス
        'nonce': web3.eth.getTransactionCount(address), # ナンス
        'gas': 2000000, # 使用するGASの上限
        'gasPrice': web3.toWei('0', 'gwei'), # 1GASの価格を設定(0に設定、手数料無しでトランザクション送信可能)
        'chainId': 12345, # ブロックチェーンのIDを指定
    }
)
# 秘密鍵でトランザクションを署名
signed_tx = Account.signTransaction(construct_tx, private_key)
# トランザクションの送信
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
# ブロックチェーン上で実装されるまで待つ
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

# 各種変数の説明
print("秘密鍵: ", private_key)
print("アドレス: ", address)
print("トランザクションハッシュ: ", tx_hash.hex())
print("実装されたコントラクトアドレス: ", tx_receipt['contractAddress'])
json.dump({"contractAddress": tx_receipt['contractAddress']}, open("./contractAddress.json", "w"))

秘密鍵:  0x253ea13322cb87d3a73f1e322f33bda37a50b527bed83ed2c57f980d89283712
アドレス:  0x81Be65E0977041aF1913c791a70AEAAF06E50F70
トランザクションハッシュ:  0x89e9d99729e0bdf55bcf02d388abc89fe38e7b4ef6cd7d0887b36acec8d143ed
実装されたコントラクトアドレス:  0x69774b66b3Fb93ECA9dDBeE92b14C721C17B4794
